In [2]:
from train import *
from networks import *
from utils import *
from dataset import *

import torch
import torchvision
import torchvision.transforms as transforms
import warnings

warnings.simplefilter("ignore")

In [3]:
print('🕺 lets gooooooooooooooooooooooooooo' if torch.cuda.is_available() else '💩 f@*# the cpu')

🕺 lets gooooooooooooooooooooooooooo


In [15]:
os.makedirs('results', exist_ok=True)
os.makedirs('weights', exist_ok=True)
os.makedirs('data/CIFAR10', exist_ok=True)
os.makedirs('data/MNIST', exist_ok=True)
os.makedirs('data/FashionMNIST', exist_ok=True)

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

datasets = {
    'MNIST': {'train':torchvision.datasets.MNIST(root='./data/MNIST', train=True, download=True, transform=transform),
              'test':torchvision.datasets.MNIST(root='./data/MNIST', train=False, download=True, transform=transform)},
    'FashionMNIST': {'train':torchvision.datasets.FashionMNIST(root='./data/FashionMNIST', train=True, download=True, transform=transform),
                     'test':torchvision.datasets.FashionMNIST(root='./data/FashionMNIST', train=False, download=True, transform=transform)},
    'CIFAR10': {'train':torchvision.datasets.CIFAR10(root='./data/CIFAR10', train=True, download=True, transform=transform),
            'test':torchvision.datasets.CIFAR10(root='./data/CIFAR10', train=False, download=True, transform=transform)},
}

In [5]:
for name, dataset in datasets.items():
    print(f"Training on {name} dataset")
    model = train_and_evaluate(name, dataset['train'], dataset['test'], device, n_epochs=10)